In [3]:
#import glob
#import sys
import time
import numpy as np
import matplotlib.pylab as plt
from scipy import misc

#import keras
from keras.layers import Dense, Flatten, Dropout, Activation
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras.models import Sequential
from keras.optimizers import SGD, Adam,Adagrad,Adadelta, Adamax,Nadam
from keras.layers import Lambda, concatenate
from keras import Model
from keras import backend as K
from keras import regularizers
from keras import callbacks
from sklearn.model_selection import train_test_split
import tensorflow as tf

Using TensorFlow backend.


In [4]:
def subtract_median(arr):
    return arr - np.median(arr)
    
def norm_image(img):
    _min = 0
    _max = 1
    img_lin = 10**(img/10) 
    X = np.log10(img_lin)
    X_std = (X - X.min()) / (X.max() - X.min())
    X_scaled = X_std * (_max - _min) + _min
    return X_scaled


""" combine multiple files """
def combine(x1,x2,y1,y2):
    xjoin = np.concatenate((x1,x2),axis=0)
    yjoin = np.concatenate((y1,y2),axis=0)
    return xjoin,yjoin
    
def norm_subtract(image):
    imag = []
    for x in range(len(image)):
        x_t = subtract_median(image[x])
        x_t = norm_image(x_t)
        ##remove shoulders
        imag.append(x_t)
    return np.array(imag)

In [5]:
batch_size = 10
num_classes = 4
epochs = 100
weight_decay = 0.0005
# input image dimensions
img_x, img_y = 30, 90
#
X = np.load('/home/nextgen/Documents/drone_detection_models/Drone_Det_CNN/Training_Data/Combined_Images.npy')
Y = np.load('/home/nextgen/Documents/drone_detection_models/Drone_Det_CNN/Training_Data/Combined_Labels.npy')

#Splitting the data set
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2)

x_train = np.expand_dims(x_train,axis = -1) 
x_test = np.expand_dims(x_test,axis = -1) 
input_shape = (img_x, img_y, 1)


# convert the data to the right type
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
y_train = y_train.astype('float32')
y_test = y_test.astype('float32')
##Convert labels to categorical 
#y_train = keras.utils.to_categorical(y_train, num_classes=4)
#y_test = keras.utils.to_categorical(y_test, num_classes=4)

print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

x_train shape: (4177, 30, 90, 1)
4177 train samples
1045 test samples


In [6]:
#Build Model
model = Sequential()
weight_decay = weight_decay

model.add(Conv2D(64, (3, 3), padding='same',
                 input_shape= input_shape,kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(Dropout(0.3))

model.add(Conv2D(64, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('relu'))
model.add(BatchNormalization())

model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(128, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(Dropout(0.4))

model.add(Conv2D(128, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('relu'))
model.add(BatchNormalization())

model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(256, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(Dropout(0.4))

model.add(Conv2D(256, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(Dropout(0.4))

model.add(Conv2D(256, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('relu'))
model.add(BatchNormalization())

model.add(MaxPooling2D(pool_size=(2, 2)))


model.add(Conv2D(512, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(Dropout(0.4))

model.add(Conv2D(512, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(Dropout(0.4))

model.add(Conv2D(512, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('relu'))
model.add(BatchNormalization())

model.add(MaxPooling2D(pool_size=(2, 2)))


model.add(Conv2D(512, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(Dropout(0.4))

model.add(Conv2D(512, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(Dropout(0.4))

model.add(Conv2D(512, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('relu'))
model.add(BatchNormalization())

#model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.5))

model.add(Flatten())
model.add(Dense(4096,kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('relu'))
model.add(BatchNormalization())

model.add(Dense(4096,kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('relu'))
model.add(BatchNormalization())

model.add(Dense(1000,kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('relu'))
model.add(BatchNormalization())

model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

In [7]:
# Optimizer options
adam = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0)
#adagrad = Adagrad(lr=0.01, epsilon=None, decay=0.0)
#adadelta = Adadelta(lr=1.0, rho=0.95, epsilon=None, decay=0.0)
#adamax = Adamax(lr=0.002, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0)
#nadam = Nadam(lr=0.002, beta_1=0.9, beta_2=0.999, epsilon=None, schedule_decay=0.004)
# Train with GPU's
#parallel_model = multi_gpu_model(model, gpus=8)
model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
"*****************************************************************************"
model.summary()
model.get_config()
model.layers[0].get_config()
model.layers[0].input_shape
model.layers[0].output_shape
model.layers[0].get_weights()
np.shape(model.layers[0].get_weights()[0])
model.layers[0].trainable

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 30, 90, 64)        640       
_________________________________________________________________
activation_1 (Activation)    (None, 30, 90, 64)        0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 30, 90, 64)        256       
_________________________________________________________________
dropout_1 (Dropout)          (None, 30, 90, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 30, 90, 64)        36928     
_________________________________________________________________
activation_2 (Activation)    (None, 30, 90, 64)        0         
_________________________________________________________________
batch_normalization_2 (Batch (None, 30, 90, 64)        256       
__________

True

In [ ]:
#Save training data to csv file
filename='VGG16_1.csv'
csv_log=callbacks.CSVLogger(filename, separator=',', append=False)
callbacks_list = [csv_log]
hist = model.fit(x_train, y_train, batch_size = batch_size,verbose=1, epochs = epochs, validation_data=(x_test, y_test),callbacks=callbacks_list)

In [ ]:
# visualizing losses and accuracy
train_loss=hist.history['loss']
val_loss=hist.history['val_loss']
train_acc=hist.history['acc']
val_acc=hist.history['val_acc']
xc=range(epochs)

plt.figure(1,figsize=(7,5))
plt.plot(xc,train_loss)
plt.plot(xc,val_loss)
plt.xlabel('num of Epochs')
plt.ylabel('loss')
plt.title('train_loss vs val_loss')
plt.grid(True)
plt.legend(['train','val'])
#print plt.style.available # use bmh, classic,ggplot for big pictures
plt.style.use(['classic'])

plt.figure(2,figsize=(7,5))
plt.plot(xc,train_acc)
plt.plot(xc,val_acc)
plt.xlabel('num of Epochs')
plt.ylabel('accuracy')
plt.title('train_acc vs val_acc')
plt.grid(True)
plt.legend(['train','val'],loc=4)
#print plt.style.available # use bmh, classic,ggplot for big pictures
plt.style.use(['classic'])

In [ ]:
# Predicting the Test set results
y_pred = model.predict(x_test)
y_pred = (y_pred > 0.5)

test_rate = np.mean(y_pred == y_test)
print ('Test accuracy: ', "{0:.2f}%".format(test_rate * 100))

In [ ]:
# save model
model.save("VGG16_Weights.h5")